<a href="https://colab.research.google.com/github/stiwari-ds/data-science-competitions/blob/main/machinehack/subscriber_prediction_hackathon/notebooks/02_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
%%capture
!pip install --upgrade optuna
!pip install --upgrade catboost

In [2]:
import os
import gc
import time
import warnings

gc.enable()
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.set_printoptions(precision=4)

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import catboost
from catboost import CatBoostClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss

In [3]:
optuna.__version__, catboost.__version__

('3.0.3', '1.1.1')

In [4]:
SEED = 2311
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)

In [5]:
DATA_URL = 'https://raw.githubusercontent.com/stiwari-ds/data-science-competitions/main/machinehack/subscriber_prediction_hackathon/data'

train = pd.read_csv(f'{DATA_URL}/processed/train.csv') #processed dataset from notebook 00
test = pd.read_csv(f'{DATA_URL}/processed/test.csv')
sample_sub = pd.read_csv(f'{DATA_URL}/raw/submission.csv')

In [6]:
TARGET = 'y_bool'

In [7]:
features = list(test.columns)
num_features = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
cat_features = [f for f in features if f not in num_features]

In [8]:
original_features = ['age', 'job', 'marital', 'education', 'default', 'balance',
                     'housing', 'loan', 'contact', 'day', 'month', 'duration', 
                     'campaign', 'pdays', 'previous', 'poutcome']

cat_only_features = ['age_bins', 'job_groups', 'marital', 'education', 'default',
                     'balance_bins', 'housing', 'loan', 'contact', 'day_bins', 
                     'month_bins', 'duration_bins', 'campaign_bins', 'pdays_bins',
                     'pdays_bool', 'previous_bins', 'previous_bool', 'poutcome']

In [9]:
train[cat_features] = train[cat_features].astype('category')
test[cat_features] = test[cat_features].astype('category')

# Baseline

In [10]:
%%time
scores = []
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
X, y = train[features], train[TARGET]
cat_features = list(X.select_dtypes(include='category').columns)
for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
    X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.loc[val_idx], y.iloc[val_idx]

    model = CatBoostClassifier(
        loss_function='Logloss',
        eval_metric='Logloss',
        use_best_model=True,
        verbose=False,
        random_state=SEED
    )
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        cat_features=cat_features,
        early_stopping_rounds=100,
        verbose=False
    )
    val_preds = model.predict_proba(X_val)[:, 1]
    score = log_loss(y_val, val_preds)
    scores.append(score)
    print(f'Fold #{fold}: ({model.best_iteration_} rounds) Logloss = {score:.5f}')
    _ = gc.collect()

print(f'\nAvg Logloss = {np.mean(scores):.5f} +/- {np.std(scores):.5f}\n')

Fold #0: (84 rounds) Logloss = 0.58051
Fold #1: (113 rounds) Logloss = 0.57998
Fold #2: (200 rounds) Logloss = 0.57857
Fold #3: (64 rounds) Logloss = 0.58136
Fold #4: (86 rounds) Logloss = 0.58065

Avg Logloss = 0.58021 +/- 0.00093

CPU times: user 1min 15s, sys: 1.73 s, total: 1min 17s
Wall time: 51.5 s


# Hyperparameter tuning

In [11]:
def objective(trial, data, base_params):

    scores = []
    X, y = data
    cat_features = list(X.select_dtypes(include='category').columns)
    
    param_grid = {
        'learning_rate': trial.suggest_float('learning_rate', 0.1, 0.5, step=0.05),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 2, 15, step=0.1),
        'depth': trial.suggest_int('depth', 3, 13),
        'rsm': trial.suggest_float('rsm', 0.5, 1.0, step=0.05), #colsample_bylevel
        'random_strength': trial.suggest_float('random_strength', 0.5, 2, step=0.05),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 5, step=0.05),
        'one_hot_max_size': trial.suggest_categorical('one_hot_max_size', [2, 5, 12]),
        'bootstrap_type': trial.suggest_categorical(
            'bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS'])
    }
    #conditional hyperparameters
    if param_grid['bootstrap_type'] == 'Bayesian':
        param_grid['bagging_temperature'] = trial.suggest_int('bagging_temperature', 0, 10)
    else:
        param_grid['subsample'] = trial.suggest_float('subsample', 0.25, 1.0, step=0.05)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = CatBoostClassifier(**base_params, **param_grid)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            cat_features=cat_features,
            early_stopping_rounds=100,
            verbose=False
        )

        val_preds = model.predict_proba(X_val)[:, 1]
        scores.append(log_loss(y_val, val_preds))
        
    return np.mean(scores)

In [12]:
def tune_params(data, base_params, n_trials, direction):
    study = optuna.create_study(
        sampler=TPESampler(seed=SEED),
        pruner=HyperbandPruner(),
        direction=direction
    )
    study.optimize(
        func=lambda trial: objective(trial, data, base_params),
        n_trials=n_trials,
        gc_after_trial=True
    )
    return study

# Cross-validation

In [13]:
def cross_validate_predict(data, model_params, n_splits=5):
    oof_preds = {}  #out-of-fold predictions on train set
    test_preds = {} #predictions on test set for each fold
    scores = [] #scores on validation set

    X, y, X_test = data
    cat_features = list(X.select_dtypes(include='category').columns)
       
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, y_train = X.loc[train_idx], y.iloc[train_idx]
        X_val, y_val = X.loc[val_idx], y.iloc[val_idx]
        
        model = CatBoostClassifier(**model_params)
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            cat_features=cat_features,
            early_stopping_rounds=100,
            verbose=False
        )

        val_preds = model.predict_proba(X_val)[:, 1]
        test_preds[f'fold{fold}']  = model.predict_proba(X_test)[:, 1]
        oof_preds.update(dict(zip(val_idx, val_preds)))

        score = log_loss(y_val, val_preds)
        scores.append(score)
        print(f'Fold #{fold}: Logloss = {score:.5f}')
        _ = gc.collect()
    print(f'Avg. Logloss = {np.mean(scores):.5f} +/- {np.std(scores):.5f}')
    
    oof_preds = pd.Series(oof_preds).sort_index()
    test_preds = pd.DataFrame.from_dict(test_preds)
    test_preds['mean'] = test_preds.mean(axis=1)

    return oof_preds, test_preds

In [14]:
def run_experiment(data, n_trials=10):
    
    X, y, X_test = data

    base_params = {
        'n_estimators': 10000,
        'loss_function': 'Logloss',
        'eval_metric': 'Logloss',
        'langevin': True,
        'use_best_model': True,
        'verbose': False,
        'random_seed': SEED
    }
        
    print(f'----------Hyperparameter tuning----------')
    start = time.time()
    study = tune_params(
        data=(X, y),
        base_params=base_params,
        n_trials=n_trials, 
        direction='minimize' #metric: logloss -> lower is better
    )
    end = time.time()
    print(f'Best trial: {study.best_trial.number} -> Best value: {study.best_value:.5f}')
    print(f'Best hyperparameters:')
    for k, v in study.best_params.items():
        print(f'{k:15} - {v}')
    print(f'[Time taken: {end - start:.2f}s]\n')
    
    print(f'-----Cross-validation and prediction-----')
    start = time.time()
    model_params = {**base_params, **study.best_params}
    oof_preds, test_preds = cross_validate_predict(data, model_params)
    end = time.time()
    print(f'[Time taken: {end - start:.2f}s]\n')

    return oof_preds, test_preds

**Trial runs**

In [15]:
optuna.logging.set_verbosity(optuna.logging.INFO)

In [17]:
%%time
op, tp = run_experiment(
    data=(train[features], train[TARGET], test[features]),
    n_trials=3
)

[I 2022-11-16 03:22:39,330] A new study created in memory with name: no-name-8a5844a2-f922-459d-82b6-34c6a58346a8


----------Hyperparameter tuning----------


[I 2022-11-16 03:22:42,147] Trial 0 finished with value: 0.7763927602072619 and parameters: {'learning_rate': 0.2, 'l2_leaf_reg': 13.0, 'depth': 5, 'rsm': 0.55, 'random_strength': 0.55, 'scale_pos_weight': 3.85, 'one_hot_max_size': 12, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1}. Best is trial 0 with value: 0.7763927602072619.
[I 2022-11-16 03:22:48,681] Trial 1 finished with value: 0.6794008203043577 and parameters: {'learning_rate': 0.45000000000000007, 'l2_leaf_reg': 14.9, 'depth': 9, 'rsm': 0.55, 'random_strength': 1.5, 'scale_pos_weight': 2.4000000000000004, 'one_hot_max_size': 12, 'bootstrap_type': 'Bayesian', 'bagging_temperature': 8}. Best is trial 1 with value: 0.6794008203043577.
[I 2022-11-16 03:22:56,904] Trial 2 finished with value: 0.8352546890797858 and parameters: {'learning_rate': 0.15000000000000002, 'l2_leaf_reg': 7.6000000000000005, 'depth': 9, 'rsm': 0.65, 'random_strength': 1.55, 'scale_pos_weight': 4.7, 'one_hot_max_size': 12, 'bootstrap_type': 'Berno

Best trial: 1 -> Best value: 0.67940
Best hyperparameters:
learning_rate   - 0.45000000000000007
l2_leaf_reg     - 14.9
depth           - 9
rsm             - 0.55
random_strength - 1.5
scale_pos_weight - 2.4000000000000004
one_hot_max_size - 12
bootstrap_type  - Bayesian
bagging_temperature - 8
[Time taken: 17.64s]

-----Cross-validation and prediction-----
Fold #0: Logloss = 0.68079
Fold #1: Logloss = 0.68067
Fold #2: Logloss = 0.67373
Fold #3: Logloss = 0.68083
Fold #4: Logloss = 0.68098
Avg. Logloss = 0.67940 +/- 0.00284
[Time taken: 7.99s]

CPU times: user 36.8 s, sys: 4.25 s, total: 41 s
Wall time: 25.6 s


In [18]:
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Exp 1

In [19]:
%%time
op1, tp1 = run_experiment(
    data=(train[features], train[TARGET], test[features]),
    n_trials=200
)

----------Hyperparameter tuning----------
Best trial: 181 -> Best value: 0.58007
Best hyperparameters:
learning_rate   - 0.45000000000000007
l2_leaf_reg     - 13.700000000000001
depth           - 3
rsm             - 0.7
random_strength - 1.9500000000000002
scale_pos_weight - 1.0
one_hot_max_size - 12
bootstrap_type  - MVS
subsample       - 0.9
[Time taken: 2822.87s]

-----Cross-validation and prediction-----
Fold #0: Logloss = 0.58065
Fold #1: Logloss = 0.57944
Fold #2: Logloss = 0.57805
Fold #3: Logloss = 0.58170
Fold #4: Logloss = 0.58051
Avg. Logloss = 0.58007 +/- 0.00124
[Time taken: 2.93s]

CPU times: user 1h 22min 59s, sys: 2min 20s, total: 1h 25min 20s
Wall time: 47min 5s


# Exp 2

In [20]:
%%time
op2, tp2 = run_experiment(
    data=(train[original_features], train[TARGET], test[original_features]),
    n_trials=200
)

----------Hyperparameter tuning----------
Best trial: 168 -> Best value: 0.58022
Best hyperparameters:
learning_rate   - 0.1
l2_leaf_reg     - 8.5
depth           - 4
rsm             - 0.8
random_strength - 0.65
scale_pos_weight - 1.0
one_hot_max_size - 5
bootstrap_type  - Bayesian
bagging_temperature - 0
[Time taken: 1832.90s]

-----Cross-validation and prediction-----
Fold #0: Logloss = 0.58001
Fold #1: Logloss = 0.58048
Fold #2: Logloss = 0.57825
Fold #3: Logloss = 0.58153
Fold #4: Logloss = 0.58085
Avg. Logloss = 0.58022 +/- 0.00110
[Time taken: 8.01s]

CPU times: user 52min 50s, sys: 1min 27s, total: 54min 18s
Wall time: 30min 40s


# Exp 3

In [21]:
%%time
op3, tp3 = run_experiment(
    data=(train[cat_only_features], train[TARGET], test[cat_only_features]),
    n_trials=200
)

----------Hyperparameter tuning----------
Best trial: 123 -> Best value: 0.58052
Best hyperparameters:
learning_rate   - 0.30000000000000004
l2_leaf_reg     - 6.6000000000000005
depth           - 3
rsm             - 0.65
random_strength - 1.2000000000000002
scale_pos_weight - 1.0
one_hot_max_size - 2
bootstrap_type  - MVS
subsample       - 0.9500000000000001
[Time taken: 2481.45s]

-----Cross-validation and prediction-----
Fold #0: Logloss = 0.58095
Fold #1: Logloss = 0.58038
Fold #2: Logloss = 0.57825
Fold #3: Logloss = 0.58188
Fold #4: Logloss = 0.58116
Avg. Logloss = 0.58052 +/- 0.00123
[Time taken: 12.32s]

CPU times: user 1h 13min 10s, sys: 1min 42s, total: 1h 14min 53s
Wall time: 41min 33s


# Submission files

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
NOTEBOOK = '02'
SUBMISSION_PATH = f'/content/drive/MyDrive/data_science_competitions/machinehack/subscriber_prediction_hackathon/submissions/nb_{NOTEBOOK}'
if not os.path.isdir(SUBMISSION_PATH):
    os.makedirs(SUBMISSION_PATH)

In [24]:
def create_submission_files(test_preds: pd.DataFrame, expt_num: int):
    for col in (test_preds.columns):
        sub = sample_sub.copy()
        sub[TARGET] = test_preds[col]
        sub.to_csv(f'{SUBMISSION_PATH}/{expt_num}_{col}.csv', index=False)

In [25]:
create_submission_files(tp1, '01')
create_submission_files(tp2, '02')
create_submission_files(tp3, '03')